How to Use This Version for Checkpoint Resume in Kaggle

Before running this notebook for the second time (to resume training from a checkpoint), follow the steps below carefully.

1. Download the latest checkpoint file
Download the file named checkpoint_latest.pth from the previous version of your notebook or experiment.

2. Upload the checkpoint to Kaggle Input Directory
Place the downloaded file inside your Kaggle input path, for example:
/kaggle/input/path1/pytorch/default/1/checkpoint_latest.pth

3. Run the following code cell before starting training
This code will copy the checkpoint file to the working directory (/kaggle/working/checkpoints) so that training can resume from the saved state.

4. Resume Training
After the checkpoint file is copied successfully, running the rest of the notebook will automatically start training from the previous checkpoint instead of starting from scratch.



## you can change the dataset and the attack type by simply changing the name in the args.py file.no need to modify anything else.

In [ ]:
import os
import shutil

# Source and destination paths
src = "/kaggle/input/path1/pytorch/default/1/checkpoint_latest.pth"
dst_dir = "/kaggle/working/checkpoints"
dst = os.path.join(dst_dir, "checkpoint_latest.pth")

# Step 1: Check if source file exists
if not os.path.exists(src):
    print(f"❌ Source file not found: {src}")
else:
    print(f"✅ Found source file: {src}")

    # Step 2: Ensure destination directory exists
    if not os.path.exists(dst_dir):
        os.makedirs(dst_dir)
        print(f"📂 Created destination directory: {dst_dir}")
    else:
        print(f"📁 Destination directory already exists: {dst_dir}")

    # Step 3: Copy the file
    shutil.copy(src, dst)
    print(f"✅ Copied file to: {dst}")

    # Step 4: List all files in destination
    files = os.listdir(dst_dir)
    if files:
        print("\n📄 Files in /kaggle/working/checkpoints:")
        for f in files:
            print(" ├──", f)
    else:
        print("⚠ Destination directory is empty (unexpected).")


In [ ]:
import os

# Target directory
base_dir = "/kaggle/working/FedBn-PyTorch"
os.makedirs(base_dir, exist_ok=True)

# Python files to create
files = ["main.py", "server.py", "client.py", "model.py", "get_data.py", "args.py"]

# Create each file if not exists
for file in files:
    file_path = os.path.join(base_dir, file)
    if not os.path.exists(file_path):
        with open(file_path, "w") as f:
            f.write(f"# {file} — auto-created placeholder\n")
        print(f" Created: {file_path}")
    else:
        print(f" Already exists: {file_path}")

print("\n Folder and files ready in:", base_dir)


In [ ]:
file_path = "/kaggle/working/FedBn-PyTorch/args.py"

new_code = '''
# ========================================
# args.py — FedBN Configuration 
# ========================================
import argparse
import torch

DATASET_CONFIGS = {
    'pathmnist': {'num_classes': 9, 'input_channels': 3},
    'tissuemnist': {'num_classes': 8, 'input_channels': 1},
    'organamnist': {'num_classes': 11, 'input_channels': 1},
    'octmnist': {'num_classes': 4, 'input_channels': 1},
}

def args_parser():
    parser = argparse.ArgumentParser(description="FedBN Config (4 Dataset Compatible)")
    
    # -------------------------------
    # Dataset
    # -------------------------------
    parser.add_argument('--dataset', type=str, default='organamnist',
                        choices=['pathmnist', 'tissuemnist', 'organamnist', 'octmnist'],
                        help='MedMNIST dataset to use')
    parser.add_argument('--use_combined', action='store_true', 
                        help='Use train+val+test combined')

    # -------------------------------
    # Federated Learning Parameters
    # -------------------------------
    parser.add_argument('--E', type=int, default=5, help='local epochs')
    parser.add_argument('--r', type=int, default=50, help='communication rounds')
    parser.add_argument('--K', type=int, default=5, help='number of clients')
    parser.add_argument('--C', type=float, default=1.0, help='client sampling rate')
    parser.add_argument('--B', type=int, default=32, help='batch size')

    # -------------------------------
    # Model Parameters
    # -------------------------------
    parser.add_argument('--clip_model', type=str, default='ViT-B/32', help='CLIP model variant')
    parser.add_argument('--freeze_clip', action='store_true', help='Freeze CLIP backbone')
    parser.add_argument('--dropout', type=float, default=0.5, help='dropout rate')

    # -------------------------------
    # Optimizer Settings
    # -------------------------------
    parser.add_argument('--lr', type=float, default=0.003, help='learning rate')
    parser.add_argument('--optimizer', type=str, default='sgd', help='optimizer type')
    parser.add_argument('--weight_decay', type=float, default=5e-4, help='weight decay')
    parser.add_argument('--momentum', type=float, default=0.9, help='momentum for SGD')

    # -------------------------------
    # FedBN Specific
    # -------------------------------
    parser.add_argument('--exclude_bn_from_agg', action='store_true',
                        help='Exclude BN params from aggregation')
    parser.add_argument('--freeze_bn', action='store_true', help='Freeze BN layers')
    
    # -------------------------------
    # Others
    # -------------------------------
    parser.add_argument('--checkpoint_dir', type=str, default='./checkpoints_fedbn_organamnist')
    parser.add_argument('--device', default=torch.device("cuda" if torch.cuda.is_available() else "cpu"))
    parser.add_argument('--dominant_ratio', type=float, default=0.7, help='dominant ratio for Non-IID split')


    # -------------------------------
    # Adversarial Attack Parameters
    # -------------------------------
    parser.add_argument('--attack_type', type=str, default='none', 
                        choices=['none', 'fgsm', 'pgd', 'cw'],
                        help='Type of adversarial attack during testing')
    parser.add_argument('--attack_epsilon', type=float, default=0.06,  # Increased for [-1,1] range
                    help='Epsilon for FGSM/PGD attacks')
    parser.add_argument('--pgd_alpha', type=float, default=0.006,  # epsilon/10
                        help='Step size for PGD')
    parser.add_argument('--pgd_iters', type=int, default=10,
                        help='Number of PGD iterations')
    parser.add_argument('--cw_c', type=float, default=1.0,
                        help='C parameter for CW attack')
    parser.add_argument('--cw_max_iter', type=int, default=100,
                        help='Max iterations for CW attack')

    args = parser.parse_args(args=[])

    if args.dataset in DATASET_CONFIGS:
        cfg = DATASET_CONFIGS[args.dataset]
        args.num_classes = cfg['num_classes']
        args.input_channels = cfg['input_channels']
    else:
        raise ValueError(f"Unsupported dataset {args.dataset}")

    return args

'''

with open(file_path, "w") as f:
    f.write(new_code)

print(" args.py written successfully!")
print(f" File path: {file_path}")


In [ ]:
file_path = "/kaggle/working/FedBn-PyTorch/get_data.py"

new_code = r'''
# ========================================
# get_data.py — Balanced Non-IID Loader (4 Dataset Compatible)
# ========================================
import os
import numpy as np
import torch
import pickle
from torch.utils.data import DataLoader, Subset, ConcatDataset
from torchvision import transforms
from medmnist.dataset import PathMNIST, TissueMNIST, OrganAMNIST, OCTMNIST

DATASET_MAP = {
    'pathmnist': PathMNIST,
    'tissuemnist': TissueMNIST,
    'organamnist': OrganAMNIST,
    'octmnist': OCTMNIST,
}

def get_transforms():
    """Unified transforms (grayscale → RGB for CLIP)"""
    train_t = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),
        transforms.Grayscale(num_output_channels=3),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
    ])
    test_t = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.Grayscale(num_output_channels=3),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
    ])
    return train_t, test_t

def balanced_noniid_split(dataset, num_clients, dominant_ratio=0.7):
    labels = np.array([dataset[i][1].item() for i in range(len(dataset))])
    n_classes = len(np.unique(labels))
    client_indices = [[] for _ in range(num_clients)]

    class_indices = {c: np.where(labels == c)[0].tolist() for c in range(n_classes)}
    for c in class_indices: np.random.shuffle(class_indices[c])

    for client_id in range(num_clients):
        dom_class = client_id % n_classes
        n_dom = int(len(class_indices[dom_class]) * dominant_ratio)
        client_indices[client_id].extend(class_indices[dom_class][:n_dom])
        class_indices[dom_class] = class_indices[dom_class][n_dom:]

    for c in range(n_classes):
        rem = class_indices[c]
        np.random.shuffle(rem)
        others = [i for i in range(num_clients) if i % n_classes != c]
        for i, idx in enumerate(rem):
            client_indices[others[i % len(others)]].append(idx)
    for i in range(num_clients):
        np.random.shuffle(client_indices[i])
    return client_indices

def load_medmnist_data(args):
    train_t, test_t = get_transforms()
    root = './data/medmnist'
    os.makedirs(root, exist_ok=True)
    Dataset = DATASET_MAP[args.dataset]

    train = Dataset(root=root, split='train', download=True, transform=train_t)
    val = Dataset(root=root, split='val', download=True, transform=train_t)
    test = Dataset(root=root, split='test', download=True, transform=test_t)
    combined = ConcatDataset([train, val])

    cache = f'./data/medmnist/client_idx_{args.dataset}_K{args.K}_dr{args.dominant_ratio}_fedbn.pkl'
    if os.path.exists(cache):
        with open(cache, 'rb') as f:
            client_idx = pickle.load(f)
    else:
        client_idx = balanced_noniid_split(combined, args.K, args.dominant_ratio)
        with open(cache, 'wb') as f:
            pickle.dump(client_idx, f)

    client_loaders = [
        DataLoader(Subset(combined, idx), batch_size=args.B, shuffle=True, num_workers=0)
        for idx in client_idx
    ]
    test_loader = DataLoader(test, batch_size=args.B, shuffle=False, num_workers=0)
    return client_loaders, test_loader

'''

with open(file_path, "w") as f:
    f.write(new_code)

print(" get_data.py created for FedBN!")


In [ ]:
file_path = "/kaggle/working/FedBn-PyTorch/client.py"

new_code = """# ========================================
# client.py — Client-side helper for FedBN (with gradient clipping)
# ========================================
import torch
from torch import nn
import torch.optim as optim
from tqdm import tqdm
import copy

class Client:
    def __init__(self, model, train_loader, device, val_loader=None,
                 lr=0.001, weight_decay=1e-4, use_fedbn=True):
        \"\"\"
        Args:
            use_fedbn (bool): If True, BN layers remain local (not aggregated)
        \"\"\"
        self.model = model.to(device)
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.device = device
        self.lr = lr
        self.weight_decay = weight_decay
        self.use_fedbn = use_fedbn

        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optim.Adam(self.model.parameters(), lr=self.lr, weight_decay=self.weight_decay)

    def compute_accuracy(self, loader):
        \"\"\"Compute accuracy on given DataLoader\"\"\"
        self.model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for images, labels in loader:
                images, labels = images.to(self.device), labels.to(self.device)
                labels = labels.squeeze()
                if labels.dim() == 0:
                    labels = labels.unsqueeze(0)
                if labels.size(0) == 0:
                    continue
                outputs = self.model(images)
                if isinstance(outputs, tuple) or hasattr(outputs, 'logits'):
                    outputs = outputs.logits
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        return 100 * correct / total if total > 0 else 0

    def train(self, epochs=1):
        \"\"\"Local training (BN layers remain local; only non-BN parameters aggregated)\"\"\"
        print()
        print(" Training client model with FedBN objective...")
        
        for epoch in range(epochs):
            self.model.train()
            total_loss = 0.0
            batch_count = 0

            pbar = tqdm(self.train_loader,
                        desc=f"  Epoch {epoch+1}/{epochs}",
                        ncols=100,
                        leave=False)

            for images, labels in pbar:
                images, labels = images.to(self.device), labels.to(self.device)
                labels = labels.squeeze()
                if labels.dim() == 0:
                    labels = labels.unsqueeze(0)
                if labels.size(0) == 0:
                    continue

                self.optimizer.zero_grad()
                outputs = self.model(images)
                if isinstance(outputs, tuple) or hasattr(outputs, 'logits'):
                    outputs = outputs.logits
                loss = self.criterion(outputs, labels)
                
                # Check for NaN and skip if found
                if torch.isnan(loss):
                    print("  NaN detected, skipping batch")
                    continue
                
                loss.backward()
                
                # Gradient clipping for stability
                torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)
                
                self.optimizer.step()

                total_loss += loss.item()
                batch_count += 1
                pbar.set_postfix({'loss': f'{loss.item():.4f}'})
            
            avg_loss = total_loss / batch_count if batch_count > 0 else 0
            train_acc = self.compute_accuracy(self.train_loader)
            log_msg = f"  Epoch {epoch+1:2d}/{epochs} | Loss: {avg_loss:.4f} | Train Acc: {train_acc:.2f}%"

            if self.val_loader is not None:
                val_acc = self.compute_accuracy(self.val_loader)
                val_loss, val_batches = 0.0, 0
                self.model.eval()
                with torch.no_grad():
                    for images, labels in self.val_loader:
                        images, labels = images.to(self.device), labels.to(self.device)
                        labels = labels.squeeze()
                        if labels.dim() == 0:
                            labels = labels.unsqueeze(0)
                        if labels.size(0) == 0:
                            continue
                        outputs = self.model(images)
                        if isinstance(outputs, tuple) or hasattr(outputs, 'logits'):
                            outputs = outputs.logits
                        loss = self.criterion(outputs, labels)
                        val_loss += loss.item()
                        val_batches += 1
                val_loss = val_loss / val_batches if val_batches > 0 else 0
                log_msg += f" | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}%"

            print(log_msg)
        
        print(" Client local training complete (FedBN)")
        print()
        return self.model.state_dict()
"""

with open(file_path, "w") as f:
    f.write(new_code)

print(" client.py updated!")

In [ ]:
file_path = "/kaggle/working/FedBn-PyTorch/model.py"

new_code = '''
# ========================================
# model.py – CLIP-based FedBN (Gradient-Compatible)
# ========================================
import torch
from torch import nn
import clip

class CLIPFedBNClassifier(nn.Module):
    def __init__(self, args):
        super().__init__()
        self.args = args
        self.clip_model, _ = clip.load(args.clip_model, device=args.device)
        
        # Freeze CLIP by default (but allow gradient computation for attacks)
        for p in self.clip_model.parameters():
            p.requires_grad = False
        
        feature_dim = self.clip_model.visual.output_dim
        
        # Classification head
        self.fc1 = nn.Linear(feature_dim, 256)
        self.bn1 = nn.BatchNorm1d(256)
        self.fc2 = nn.Linear(256, args.num_classes)
        self.bn2 = nn.BatchNorm1d(args.num_classes)
        self.relu = nn.ReLU()
        self.drop = nn.Dropout(args.dropout)
    
    def forward(self, x, enable_grad=False):
        """
        Forward pass with optional gradient computation
        
        Args:
            x: Input images
            enable_grad: If True, compute gradients through CLIP (for attacks)
        """
        if enable_grad:
            # For adversarial attacks - enable gradients through CLIP
            # This allows attacks to perturb inputs effectively
            img_feat = self.clip_model.encode_image(x).float()
        else:
            # Normal training/inference - no gradients through CLIP
            with torch.no_grad():
                img_feat = self.clip_model.encode_image(x).float()
        
        # Classification head (always has gradients during training)
        x = self.fc1(img_feat)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.bn2(x)
        
        return x
    
    def get_bn_params(self):
        """
        Return list of BatchNorm parameter names
        These should be excluded from FedBN aggregation
        """
        bn_params = []
        for name, module in self.named_modules():
            if isinstance(module, (nn.BatchNorm1d, nn.BatchNorm2d)):
                # Get all BN-related parameters
                for param_name in ['weight', 'bias', 'running_mean', 'running_var', 
                                   'num_batches_tracked']:
                    full_name = f"{name}.{param_name}"
                    bn_params.append(full_name)
        return bn_params
    
    def extract_features(self, x):
        """
        Extract CLIP features (useful for analysis)
        """
        with torch.no_grad():
            return self.clip_model.encode_image(x).float()
'''

with open(file_path, "w") as f:
    f.write(new_code)

print(" model.py fixed!")
print(f" File path: {file_path}")

In [ ]:
file_path = "/kaggle/working/FedBn-PyTorch/attacks.py"

new_code = """
# ========================================
# attacks.py – Fixed Adversarial Attacks for FedBN
# ========================================
import torch
import torch.nn as nn

class AdversarialAttacks:
    def __init__(self, model, device):
        self.model = model
        self.device = device
        self.criterion = nn.CrossEntropyLoss()
    
    def fgsm_attack(self, images, labels, epsilon=0.03):
       
        self.model.eval()
        
        # Clone and enable gradients
        images = images.clone().detach().to(self.device)
        labels = labels.to(self.device)
        images.requires_grad = True
        
        # Forward pass with gradient tracking
        outputs = self.model(images, enable_grad=True)
        loss = self.criterion(outputs, labels)
        
        # Compute gradients
        self.model.zero_grad()
        loss.backward()
        
        # Check if gradients exist
        if images.grad is None:
            print("    No gradients computed - attack failed")
            return images.detach()
        
        # Create perturbation
        # Scale epsilon for normalized space [-1, 1]
        # Since your images are in [-1, 1], epsilon acts differently than [0, 1]
        perturbation = epsilon * images.grad.sign()
        
        # Create adversarial images
        adv_images = images + perturbation
        adv_images = torch.clamp(adv_images, -1, 1)  # Match your normalization
        
        return adv_images.detach()
    
    def pgd_attack(self, images, labels, epsilon=0.03, alpha=0.007, iters=10):
        
        self.model.eval()
        
        images = images.clone().detach().to(self.device)
        labels = labels.to(self.device)
        
        # Initialize with random perturbation (recommended for PGD)
        adv_images = images.clone()
        adv_images = adv_images + torch.empty_like(adv_images).uniform_(-epsilon, epsilon)
        adv_images = torch.clamp(adv_images, -1, 1)
        
        for i in range(iters):
            adv_images.requires_grad = True
            
            # Forward pass
            outputs = self.model(adv_images, enable_grad=True)
            loss = self.criterion(outputs, labels)
            
            # Compute gradients
            self.model.zero_grad()
            if adv_images.grad is not None:
                adv_images.grad.zero_()
            
            loss.backward()
            
            # Check gradients
            if adv_images.grad is None:
                print(f"   ⚠️ No gradients at iteration {i+1}")
                break
            
            # Update with gradient ascent
            with torch.no_grad():
                # Take step in direction of gradient
                adv_images = adv_images.detach() + alpha * adv_images.grad.sign()
                
                # Project back to epsilon ball around original image
                perturbation = torch.clamp(adv_images - images, -epsilon, epsilon)
                adv_images = images + perturbation
                
                # Ensure valid range
                adv_images = torch.clamp(adv_images, -1, 1)
        
        return adv_images.detach()
    
    def cw_attack(self, images, labels, c=1.0, kappa=0, max_iter=100, learning_rate=0.01):
       
        self.model.eval()
        
        images = images.clone().detach().to(self.device)
        labels = labels.to(self.device)
        batch_size = images.shape[0]
        
        # Use tanh space for better optimization
        # Transform: x = 0.5 * (tanh(w) + 1) * 2 - 1  for [-1,1] range
        w = torch.zeros_like(images, requires_grad=True, device=self.device)
        optimizer = torch.optim.Adam([w], lr=learning_rate)
        
        best_adv = images.clone()
        best_loss = float('inf') * torch.ones(batch_size, device=self.device)
        
        for step in range(max_iter):
            # Transform w to valid image range
            adv_images = torch.tanh(w) * 1.0  # Maps to [-1, 1]
            
            # Forward pass
            outputs = self.model(adv_images, enable_grad=True)
            
            # CW loss formulation
            # f(x) = max(Z_target - max(Z_other), -kappa)
            real = outputs.gather(1, labels.unsqueeze(1)).squeeze(1)
            
            # Get second highest logit
            other = outputs.clone()
            other.scatter_(1, labels.unsqueeze(1), -float('inf'))
            other_max = other.max(1)[0]
            
            # Loss: want other_max > real (misclassification)
            f_loss = torch.clamp(real - other_max + kappa, min=0)
            
            # L2 distance loss
            l2_dist = torch.sum((adv_images - images) ** 2, dim=[1, 2, 3])
            
            # Combined loss
            loss = (c * f_loss + l2_dist).sum()
            
            # Optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            # Track best adversarial examples
            with torch.no_grad():
                pred = outputs.argmax(1)
                successful = (pred != labels)
                
                for i in range(batch_size):
                    if successful[i] and l2_dist[i] < best_loss[i]:
                        best_loss[i] = l2_dist[i]
                        best_adv[i] = adv_images[i]
        
        return best_adv.detach()


def test_attacks_quickly(model, test_loader, device, args):
   
    attacker = AdversarialAttacks(model, device)
    
    # Get one batch
    images, labels = next(iter(test_loader))
    images, labels = images.to(device), labels.squeeze().to(device)
    
    print("\\n" + "="*60)
    print(" QUICK ATTACK TEST")
    print("="*60)
    
    # Test clean accuracy
    model.eval()
    with torch.no_grad():
        clean_outputs = model(images)
        clean_pred = clean_outputs.argmax(1)
        clean_acc = (clean_pred == labels).float().mean().item() * 100
    
    print(f"Clean Accuracy: {clean_acc:.2f}%")
    
    # Test FGSM
    print("\\n Testing FGSM...")
    fgsm_images = attacker.fgsm_attack(images, labels, epsilon=args.attack_epsilon)
    with torch.no_grad():
        fgsm_outputs = model(fgsm_images)
        fgsm_pred = fgsm_outputs.argmax(1)
        fgsm_acc = (fgsm_pred == labels).float().mean().item() * 100
    print(f"FGSM Accuracy: {fgsm_acc:.2f}% (Drop: {clean_acc - fgsm_acc:.2f}%)")
    
    # Test PGD
    print("\\n Testing PGD...")
    pgd_images = attacker.pgd_attack(images, labels, 
                                     epsilon=args.attack_epsilon,
                                     alpha=args.pgd_alpha,
                                     iters=args.pgd_iters)
    with torch.no_grad():
        pgd_outputs = model(pgd_images)
        pgd_pred = pgd_outputs.argmax(1)
        pgd_acc = (pgd_pred == labels).float().mean().item() * 100
    print(f"PGD Accuracy: {pgd_acc:.2f}% (Drop: {clean_acc - pgd_acc:.2f}%)")
    
    # Perturbation analysis
    fgsm_pert = (fgsm_images - images).abs().mean().item()
    pgd_pert = (pgd_images - images).abs().mean().item()
    
    print("\\n Perturbation Statistics:")
    print(f"FGSM - Mean: {fgsm_pert:.6f}, Max: {(fgsm_images - images).abs().max().item():.6f}")
    print(f"PGD  - Mean: {pgd_pert:.6f}, Max: {(pgd_images - images).abs().max().item():.6f}")
    print("="*60)
"""

with open(file_path, "w") as f:
    f.write(new_code)

print(" attacks.py updated (clean implementation)!")
print(f" File path: {file_path}")

In [ ]:
file_path = "/kaggle/working/FedBn-PyTorch/server.py"

new_code = '''
# ========================================
# server.py – FedBN Server (FIXED Attack Testing)
# ========================================
import torch
import copy
import os
from tqdm import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np
from client import Client
from model import CLIPFedBNClassifier
from get_data import load_medmnist_data
from attacks import AdversarialAttacks

class FedBNServer:
    def __init__(self, args):
        self.args = args
        self.device = args.device
        
        # Initialize global model
        self.global_model = CLIPFedBNClassifier(args).to(self.device)
        
        # Get BN params to exclude from aggregation
        self.bn_params = set(self.global_model.get_bn_params())
        print(f" Excluded BN parameters: {len(self.bn_params)} params")
        
        # Load data
        print("\\n Loading dataset...")
        self.client_loaders, self.test_loader = load_medmnist_data(args)
        print(f" Loaded {len(self.client_loaders)} clients")
        
        # Training state
        self.current_round = 0
        self.best_global_acc = 0
        self.history = {
            'rounds': [],
            'avg_accuracy': [],
            'best_accuracy': []
        }
        
        # Create checkpoint directory
        os.makedirs(args.checkpoint_dir, exist_ok=True)
        
        # Try to resume from checkpoint
        self.resume_from_checkpoint()
    
    def resume_from_checkpoint(self):
        """Load checkpoint if exists"""
        checkpoint_path = os.path.join(self.args.checkpoint_dir, 'checkpoint_latest.pth')
        
        if os.path.exists(checkpoint_path):
            print(f"\\n Found checkpoint at {checkpoint_path}. Resuming training...")
            try:
                print(f"\\n Loading checkpoint: {checkpoint_path}")
                checkpoint = torch.load(checkpoint_path, map_location=self.device, weights_only=False)
                
                self.global_model.load_state_dict(checkpoint['server_state_dict'])
                self.current_round = checkpoint['round']
                self.best_global_acc = checkpoint.get('best_global_acc', 0)
                self.history = checkpoint.get('history', self.history)
                
                print(f" Resumed from Round {self.current_round} | Best Acc: {self.best_global_acc:.2f}%")
            except Exception as e:
                print(f" Error loading checkpoint: {e}")
                print("Starting fresh training...")
                self.current_round = 0
    
    def fedbn_aggregate(self, client_states):
        """
        FedBN aggregation: Average only non-BN parameters
        BN statistics remain local to each client
        """
        global_dict = self.global_model.state_dict()
        
        # Average non-BN parameters
        for key in global_dict.keys():
            if key not in self.bn_params:
                # Average this parameter across clients
                global_dict[key] = torch.stack([
                    client_states[i][key].float() for i in range(len(client_states))
                ]).mean(0)
        
        self.global_model.load_state_dict(global_dict)
    
    def test_global_model(self):
        """
        Test global model with optional adversarial attacks
         FIXED: Each attack works on CLEAN images independently
        """
        self.global_model.eval()
        
        all_preds = []
        all_labels = []
        total_loss = 0
        total_samples = 0
        
        criterion = torch.nn.CrossEntropyLoss()
        
        # Initialize attacker if needed
        attacker = None
        if self.args.attack_type != 'none':
            attacker = AdversarialAttacks(self.global_model, self.device)
            print(f"\\n  Testing with {self.args.attack_type.upper()} attack (ε={self.args.attack_epsilon})")
        
        # Use appropriate context based on attack type
        context_manager = torch.no_grad() if self.args.attack_type == 'none' else torch.enable_grad()
        
        with context_manager:
            for images, labels in tqdm(self.test_loader, desc="  Testing", leave=False, ncols=100):
                #  Store ORIGINAL clean images
                original_images = images.clone()
                
                images, labels = images.to(self.device), labels.squeeze().to(self.device)
                
                #  Apply attack on ORIGINAL clean images (not modified images!)
                if self.args.attack_type == 'fgsm':
                    images = attacker.fgsm_attack(original_images.to(self.device), labels, 
                                                   self.args.attack_epsilon)
                elif self.args.attack_type == 'pgd':
                    images = attacker.pgd_attack(
                        original_images.to(self.device), labels,  
                        epsilon=self.args.attack_epsilon,
                        alpha=self.args.pgd_alpha,
                        iters=self.args.pgd_iters
                    )
                elif self.args.attack_type == 'cw':
                    images = attacker.cw_attack(
                        original_images.to(self.device), labels,  
                        c=self.args.cw_c,
                        max_iter=self.args.cw_max_iter
                    )
                
                # Get predictions
                with torch.no_grad():
                    outputs = self.global_model(images)
                    loss = criterion(outputs, labels)
                    _, predicted = torch.max(outputs, 1)
                
                all_preds.extend(predicted.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
                total_loss += loss.item() * labels.size(0)
                total_samples += labels.size(0)
        
        # Calculate metrics
        accuracy = 100.0 * np.mean(np.array(all_preds) == np.array(all_labels))
        avg_loss = total_loss / total_samples
        
        precision = precision_score(all_labels, all_preds, average='weighted', zero_division=0)
        recall = recall_score(all_labels, all_preds, average='weighted', zero_division=0)
        f1 = f1_score(all_labels, all_preds, average='weighted', zero_division=0)
        
        # RMSE
        rmse = np.sqrt(np.mean((np.array(all_preds) - np.array(all_labels)) ** 2))
        
        return {
            'accuracy': accuracy,
            'loss': avg_loss,
            'precision': precision,
            'recall': recall,
            'f1': f1,
            'rmse': rmse
        }
    
    def save_checkpoint(self, round_num, is_best=False):
        """Save checkpoint"""
        checkpoint = {
            'round': round_num,
            'server_state_dict': self.global_model.state_dict(),
            'best_global_acc': self.best_global_acc,
            'history': self.history,
            'args': vars(self.args)
        }
        
        # Save latest checkpoint
        latest_path = os.path.join(self.args.checkpoint_dir, 'checkpoint_latest.pth')
        torch.save(checkpoint, latest_path)
        
        # Save best checkpoint if applicable
        if is_best:
            best_path = os.path.join(self.args.checkpoint_dir, 'checkpoint_best.pth')
            torch.save(checkpoint, best_path)
            print(f"   Saved best checkpoint (Acc: {self.best_global_acc:.2f}%)")
    
    def run(self):
        """Run federated learning"""
        print("\\n" + "="*60)
        print(" STARTING FEDBN TRAINING")
        print("="*60)
        
        for round_num in range(self.current_round + 1, self.args.r + 1):
            print(f"\\n{'='*60}")
            print(f" COMMUNICATION ROUND {round_num}/{self.args.r}")
            print('='*60)
            
            # Select clients
            num_selected = max(1, int(self.args.C * self.args.K))
            selected_clients = np.random.choice(self.args.K, num_selected, replace=False)
            print(f" Selected {num_selected} clients: {selected_clients.tolist()}")
            
            # Collect client updates
            client_states = []
            
            for client_id in selected_clients:
                print(f"\\n Training Client {client_id+1}/{self.args.K}")
                
                # Create client with global model
                client_model = copy.deepcopy(self.global_model)
                client = Client(
                    model=client_model,
                    train_loader=self.client_loaders[client_id],
                    device=self.device,
                    lr=self.args.lr,
                    weight_decay=self.args.weight_decay,
                    use_fedbn=True
                )
                
                # Train locally
                updated_state = client.train(epochs=self.args.E)
                client_states.append(updated_state)
            
            # FedBN Aggregation (exclude BN params)
            print(f"\\n Aggregating {len(client_states)} client models (FedBN)...")
            self.fedbn_aggregate(client_states)
            
            # Test global model
            print("\\n Testing global model...")
            metrics = self.test_global_model()
            
            # Update history
            self.history['rounds'].append(r+1)
            self.history['avg_accuracy'].append(avg_acc)
            self.history['precision'] = self.history.get('precision', []) + [metrics['precision']]
            self.history['recall'] = self.history.get('recall', []) + [metrics['recall']]
            self.history['f1'] = self.history.get('f1', []) + [metrics['f1']]
            self.history['rmse'] = self.history.get('rmse', []) + [metrics['rmse']]
            
            # Check if best
            is_best = metrics['accuracy'] > self.best_global_acc
            if is_best:
                self.best_global_acc = metrics['accuracy']
            
            # Print results
            print(f"\\n Round {round_num} Results:")
            print(f"   Accuracy:  {metrics['accuracy']:.2f}%")
            print(f"   Loss:      {metrics['loss']:.4f}")
            print(f"   Precision: {metrics['precision']:.4f}")
            print(f"   Recall:    {metrics['recall']:.4f}")
            print(f"   F1-Score:  {metrics['f1']:.4f}")
            print(f"   Best Ever: {self.best_global_acc:.2f}%")
            
            # Save checkpoint
            self.current_round = round_num
            self.save_checkpoint(round_num, is_best)
        
        print("\\n" + "="*60)
        print("✅ FEDBN TRAINING COMPLETED")
        print("="*60)
        
        return self.global_model
'''

with open(file_path, "w") as f:
    f.write(new_code)

print(" server.py FIXED - Each attack now works on clean images!")
print(f" File path: {file_path}")

In [ ]:
file_path = "/kaggle/working/FedBn-PyTorch/main.py"

new_code = '''# ========================================
# main.py — Run FedBN with CLIP model (Auto Resume)
# ========================================
from args import args_parser
from server import FedBNServer
import torch
import os
import json
import matplotlib.pyplot as plt

def plot_training_history(history, save_path):
    """Plot training accuracy curves"""
    if not history['rounds']:
        print("  No training history to plot")
        return
    
    plt.figure(figsize=(12, 5))
    
    # Plot 1: Average Accuracy
    plt.subplot(1, 2, 1)
    plt.plot(history['rounds'], history['avg_accuracy'], 'b-', label='Avg Accuracy', linewidth=2)
    plt.plot(history['rounds'], history['best_accuracy'], 'r--', label='Best Accuracy', linewidth=2)
    plt.xlabel('Communication Round')
    plt.ylabel('Accuracy (%)')
    plt.title('Federated Learning Accuracy Progress')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Plot 2: Accuracy Improvement
    plt.subplot(1, 2, 2)
    if len(history['avg_accuracy']) > 1:
        improvements = [history['avg_accuracy'][i] - history['avg_accuracy'][i-1] 
                        for i in range(1, len(history['avg_accuracy']))]
        plt.bar(history['rounds'][1:], improvements, alpha=0.7)
        plt.axhline(y=0, color='r', linestyle='-', linewidth=0.5)
        plt.xlabel('Communication Round')
        plt.ylabel('Accuracy Change (%)')
        plt.title('Round-to-Round Accuracy Change')
        plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()
    print(f' Training plot saved at: {save_path}')


def test_with_all_attacks(server, args):
 
    attacks = ['none', 'fgsm', 'pgd', 'cw']
    results = {}
    
    print('\\n' + '='*60)
    print('  ADVERSARIAL ROBUSTNESS TESTING')
    print('='*60)
    
    for attack in attacks:
        original_attack = args.attack_type
        args.attack_type = attack
        
        print(f"\\n{'='*60}")
        print(f" Testing with {attack.upper()} attack")
        print('='*60)
        
        metrics = server.test_global_model()
        results[attack] = metrics
        
        print(f"\\n Results:")
        print(f"    Accuracy:  {metrics['accuracy']:.2f}%")
        print(f"    Precision: {metrics['precision']:.4f}")
        print(f"    Recall:    {metrics['recall']:.4f}")
        print(f"    F1-Score:  {metrics['f1']:.4f}")
        print(f"    RMSE:      {metrics['rmse']:.4f}")
    
    args.attack_type = original_attack
    
    # Save results
    results_path = os.path.join(args.checkpoint_dir, 'attack_results.json')
    with open(results_path, 'w') as f:
        json.dump(results, f, indent=4)
    print(f"\\n Attack results saved at: {results_path}")
    
    return results

def main():
    # Load arguments
    args = args_parser()
    
    # Print configuration
    print('\\n' + '='*60)
    print(' FEDBN TRAINING CONFIGURATION (CLIP Model)')
    print('='*60)
    print(f'Device: {args.device}')
    print(f'Clients: {args.K} | Rounds: {args.r} | Local Epochs: {args.E}')
    print(f'Batch Size: {args.B} | Learning Rate: {args.lr}')
    print(f'Non-IID: {args.dominant_ratio*100:.0f}% dominant class per client')
    print(f'Checkpoint Directory: {args.checkpoint_dir}')
    print('='*60)
    
    # Check if checkpoint exists (just for info - server will auto-resume)
    latest_checkpoint = os.path.join(args.checkpoint_dir, 'checkpoint_latest.pth')
    if os.path.exists(latest_checkpoint):
        print('\\n' + '='*60)
        print(' CHECKPOINT FOUND - Will auto-resume')
        print('='*60)
        try:
            checkpoint = torch.load(latest_checkpoint, map_location=args.device, weights_only=False)
            completed_rounds = checkpoint.get('round', 0)
            best_acc = checkpoint.get('best_global_acc', 0)
            print(f' Checkpoint Details:')
            print(f'   - Completed Rounds: {completed_rounds}/{args.r}')
            print(f'   - Best Accuracy: {best_acc:.2f}%')
        except Exception as e:
            print(f'  Error reading checkpoint: {e}')
    else:
        print('\\n' + '='*60)
        print(' No checkpoint found. Starting fresh training...')
        print('='*60)
    
    # Initialize FedBN server (auto-resumes if checkpoint exists)
    server = FedBNServer(args)
    
      # Check if training is complete
    if server.current_round == args.r:
        print('\\n Training already complete!')
        print(f' Running adversarial robustness testing...')
        attack_results = test_with_all_attacks(server, args)
        
        # Print final attack comparison
        print('\\n' + '='*60)
        print(' ATTACK RESULTS COMPARISON')
        print('='*60)
        print(f"{'Attack Type':<15} {'Accuracy':<10} {'Precision':<10} {'Recall':<10} {'F1-Score'}")
        print('-'*60)
        for attack, metrics in attack_results.items():
            print(f"{attack.upper():<15} {metrics['accuracy']:>6.2f}%    {metrics['precision']:>7.4f}    {metrics['recall']:>7.4f}    {metrics['f1']:>7.4f}")
        print('='*60)
    else:
        # Run federated training
        final_model = server.run()
        
        # Save final global model
        final_model_path = os.path.join(args.checkpoint_dir, 'final_global_model.pth')
        torch.save(final_model.state_dict(), final_model_path)
        print(f'\\n Final global model saved at: {final_model_path}')
        
        # Save training history
        history_path = os.path.join(args.checkpoint_dir, 'training_history.json')
        with open(history_path, 'w') as f:
            json.dump(server.history, f, indent=4)
        print(f' Training history saved at: {history_path}')
        
        # Plot training curves
        plot_path = os.path.join(args.checkpoint_dir, 'training_plot.png')
        plot_training_history(server.history, plot_path)
        
        # Print summary
        print('\\n' + '='*60)
        print(' TRAINING SUMMARY')
        print('='*60)
        print(f' Best Global Accuracy: {server.best_global_acc:.2f}%')
        print(f' Final Round Accuracy: {server.history[\"avg_accuracy\"][-1]:.2f}%')
        print(f' Total Improvement: {server.history[\"avg_accuracy\"][-1] - server.history[\"avg_accuracy\"][0]:.2f}%')
        print(f' All checkpoints saved in: {args.checkpoint_dir}')
        print('='*60)
        
        # Test with attacks after training
        print('\\n Testing adversarial robustness...')
        attack_results = test_with_all_attacks(server, args)
        
        # Print final attack comparison
        print('\\n' + '='*60)
        print(' ATTACK RESULTS COMPARISON')
        print('='*60)
        print(f"{'Attack Type':<15} {'Accuracy':<10} {'Precision':<10} {'Recall':<10} {'F1-Score'}")
        print('-'*60)
        for attack, metrics in attack_results.items():
            print(f"{attack.upper():<15} {metrics['accuracy']:>6.2f}%    {metrics['precision']:>7.4f}    {metrics['recall']:>7.4f}    {metrics['f1']:>7.4f}")
        print('='*60)


if __name__ == '__main__':
    main()
'''

with open(file_path, "w") as f:
    f.write(new_code)

print(" FedBN main.py saved successfully!")

In [ ]:
!pip install medmnist --quiet

In [ ]:
!pip install git+https://github.com/openai/CLIP.git

In [ ]:
file_path = "/kaggle/working/FedBn-PyTorch/quick_attack_test.py"

new_code = """
# ========================================
# quick_attack_test.py — Quick Attack Test for FedBN
# ========================================
from args import args_parser
from model import CLIPFedBNClassifier  
from attacks import AdversarialAttacks
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Subset
import numpy as np
import matplotlib.pyplot as plt
import os
from get_data import get_transforms
from medmnist import PathMNIST, TissueMNIST, OrganAMNIST, OCTMNIST

DATASET_MAP = {
    'pathmnist': PathMNIST,
    'tissuemnist': TissueMNIST,
    'organamnist': OrganAMNIST,
    'octmnist': OCTMNIST
}

def quick_attack_test():
    # Load arguments
    args = args_parser()
    
    print('\\n' + '='*60)
    print('⚡ QUICK ATTACK TEST (10 images) - FedBN')
    print('='*60)
    print(f'Dataset: {args.dataset.upper()}')
    print(f'Device: {args.device}')
    print('='*60)
    
    # Load model - FedBN model doesn't need 'name' parameter
    print('\\n Loading FedBN model...')
    model = CLIPFedBNClassifier(args).to(args.device)  #  No 'name' parameter
    
    # Try to load checkpoint
    checkpoint_path = os.path.join(args.checkpoint_dir, "checkpoint_latest.pth")
    try:
        if os.path.exists(checkpoint_path):
            checkpoint = torch.load(checkpoint_path, map_location=args.device, weights_only=False)
            model.load_state_dict(checkpoint['server_state_dict'])
            print(f" Loaded checkpoint from Round {checkpoint.get('round', 'unknown')}")
            print(f"   Best Accuracy: {checkpoint.get('best_global_acc', 0):.2f}%")
        else:
            print(f"  No checkpoint found at: {checkpoint_path}")
            print("   Using random weights (model not trained yet)")
    except Exception as e:
        print(f"  Error loading checkpoint: {e}")
        print("   Using random weights")
    
    model.eval()
    
    # Load test data
    print('\\n Loading test images...')
    _, test_transform = get_transforms()
    
    # Create data directory if not exists
    data_root = './data/medmnist'
    os.makedirs(data_root, exist_ok=True)
    
    DatasetClass = DATASET_MAP[args.dataset]
    test_dataset = DatasetClass(
        root=data_root, 
        split='test', 
        download=True, 
        transform=test_transform
    )
    
    # Select 10 random images
    num_images = 10
    np.random.seed(42)
    indices = np.random.choice(len(test_dataset), num_images, replace=False)
    subset = Subset(test_dataset, indices)
    test_loader = DataLoader(subset, batch_size=num_images, shuffle=False)
    
    # Get batch
    images, labels = next(iter(test_loader))
    images = images.to(args.device)
    labels = labels.squeeze().to(args.device)
    
    print(f" Loaded {num_images} test images")
    print(f" True labels: {labels.cpu().numpy()}")
    
    # Initialize attacker
    attacker = AdversarialAttacks(model, args.device)
    
    # Define attacks to test
    attacks = {
        'Clean (No Attack)': {
            'fn': None,
            'params': {}
        },
        'FGSM': {
            'fn': attacker.fgsm_attack,
            'params': {'epsilon': args.attack_epsilon}
        },
        'PGD': {
            'fn': attacker.pgd_attack,
            'params': {
                'epsilon': args.attack_epsilon,
                'alpha': args.pgd_alpha,
                'iters': args.pgd_iters
            }
        },
        'CW': {
            'fn': attacker.cw_attack,
            'params': {
                'c': args.cw_c,
                'max_iter': 50  # Reduced for quick test
            }
        }
    }
    
    print('\\n' + '='*60)
    print('  TESTING ADVERSARIAL ATTACKS')
    print('='*60)
    
    results = {}
    adv_images_dict = {}
    
    for attack_name, attack_config in attacks.items():
        print(f'\\n Testing: {attack_name}')
        
        if attack_config['fn'] is None:
            # Clean images
            test_images = images
            print('   Using clean images...')
        else:
            # Apply attack
            print(f'   Generating adversarial examples...')
            print(f'   Parameters: {attack_config["params"]}')
            test_images = attack_config['fn'](images, labels, **attack_config['params'])
        
        adv_images_dict[attack_name] = test_images
        
        # Get predictions
        with torch.no_grad():
            outputs = model(test_images)
            # FedBN model returns logits directly (no .logits attribute)
            _, predicted = torch.max(outputs, 1)
        
        # Calculate metrics
        correct = (predicted == labels).sum().item()
        accuracy = 100.0 * correct / num_images
        
        results[attack_name] = {
            'predictions': predicted.cpu().numpy(),
            'accuracy': accuracy,
            'correct': correct
        }
        
        print(f'    Predictions: {predicted.cpu().numpy()}')
        print(f'    Accuracy: {accuracy:.1f}% ({correct}/{num_images})')
    
    # Print comparison table
    print('\\n' + '='*60)
    print(' ATTACK RESULTS COMPARISON')
    print('='*60)
    print(f"{'Attack Type':<20} {'Accuracy':<12} {'Correct':<10} {'Drop'}")
    print('-'*60)
    
    clean_acc = results['Clean (No Attack)']['accuracy']
    for attack_name, result in results.items():
        acc = result['accuracy']
        drop = clean_acc - acc if attack_name != 'Clean (No Attack)' else 0
        print(f"{attack_name:<20} {acc:>6.1f}%      {result['correct']:>2}/{num_images}      {drop:>5.1f}%")
    print('='*60)
    
    # Visualize results
    print('\\n Creating visualization...')
    fig, axes = plt.subplots(2, 5, figsize=(20, 8))
    fig.suptitle('FedBN Adversarial Attack Visualization (First 5 Images)', fontsize=16, weight='bold')
    
    attack_names = list(attacks.keys())
    num_to_show = 5
    
    for img_idx in range(num_to_show):
        for attack_idx, attack_name in enumerate(attack_names):
            ax = axes[attack_idx, img_idx]
            
            # Get image
            img = adv_images_dict[attack_name][img_idx]
            
            # Denormalize
            img_np = img.cpu().detach().numpy()
            img_np = np.transpose(img_np, (1, 2, 0))
            img_np = (img_np * 0.5) + 0.5
            img_np = np.clip(img_np, 0, 1)
            
            # Get prediction
            pred = results[attack_name]['predictions'][img_idx]
            true_label = labels[img_idx].item()
            
            # Show image
            ax.imshow(img_np)
            ax.axis('off')
            
            # Title with color
            if img_idx == 0:
                title = f'{attack_name}\\nPred:{pred} True:{true_label}'
            else:
                title = f'Pred:{pred}\\nTrue:{true_label}'
            
            color = 'green' if pred == true_label else 'red'
            ax.set_title(title, color=color, fontsize=9, weight='bold')
    
    plt.tight_layout()
    
    # Save visualization
    os.makedirs(args.checkpoint_dir, exist_ok=True)
    save_path = os.path.join(args.checkpoint_dir, 'quick_attack_test.png')
    plt.savefig(save_path, dpi=150, bbox_inches='tight')
    print(f' Visualization saved: {save_path}')
    plt.show()
    
    # Perturbation analysis
    print('\\n' + '='*60)
    print(' PERTURBATION ANALYSIS')
    print('='*60)
    
    clean_img = images[0]
    for attack_name in ['FGSM', 'PGD', 'CW']:
        if attack_name in adv_images_dict:
            adv_img = adv_images_dict[attack_name][0]
            perturbation = (adv_img - clean_img).abs()
            
            print(f'\\n{attack_name}:')
            print(f'   Mean perturbation: {perturbation.mean().item():.6f}')
            print(f'   Max perturbation:  {perturbation.max().item():.6f}')
            print(f'   L2 norm:           {torch.norm(perturbation).item():.6f}')
    
    print('\\n' + '='*60)
    print(' Quick attack test completed!')
    print('='*60)
    
    return results

if __name__ == '__main__':
    quick_attack_test()
"""

with open(file_path, "w") as f:
    f.write(new_code)

print(" quick_attack_test.py updated for FedBN!")
print(f" File path: {file_path}")

In [ ]:
!python /kaggle/working/FedBn-PyTorch/main.py